In [ ]:
# import necessary modules
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import gc

import seaborn as sns
sns.set(style = 'whitegrid', color_codes = True)
%matplotlib inline

#For statistical tests
import scipy.stats as st

#For formula notation (similar to R)
import statsmodels.formula.api as smf

In [ ]:
# ## CAUTION!! : load full data
# # load complete train data
# types_dict = {'id': 'int32',
#              'item_nbr': 'int32',
#              'store_nbr': 'int8',
#              'unit_sales': 'float32',
#              'onpromotion': 'bool'}

# train_df_full = pd.read_csv('data/train.csv',low_memory=True,  parse_dates=['date'],dtype=types_dict)

# ## save in feather for easy reload 
# train_df_full.to_feather('data/train.feat')


# Preprocessing

In [ ]:
train_df_full = pd.read_feather('data/train.feat')

## Load Info

### Load Time Info
1. check the time distribution
2. select subset of time to speed up processing

In [ ]:
# extract the time feature
train_df_full['year'] = train_df_full['date'].dt.year
train_df_full['month'] = train_df_full['date'].dt.month
train_df_full['day'] = train_df_full['date'].dt.day
train_df_full['quater'] = train_df_full['date'].dt.quarter
# check the distribution
print(np.unique(train_df_full.year))
print(np.unique(train_df_full.month))
print(np.unique(train_df_full.day))

In [ ]:
# if fully loaded
# select August Data and date > 15 as subdataset
train_df1 = train_df_full[train_df_full.year == 2017]
train_df2 = train_df_full[train_df_full.year == 2016];
train_df2 = train_df2[train_df2.month >=8]
train_df = pd.concat([train_df1,train_df2])
display(train_df.head(2))
# clean unuseful raw data
del train_df_full;del train_df1; del train_df2; gc.collect();

### Load item, store, oil and transaction info


In [ ]:
# load store info
store_df = pd.read_csv('data/stores.csv',dtype = {'type':'category','cluster':'category','city':'category','state':'category'})
# load item info
items_df = pd.read_csv('data/items.csv',dtype = {'family':'category','perishable':bool,'class':'category'})
# load oil info
oil_df = pd.read_csv('data/oil.csv')
#load transacrion
trans_df = pd.read_csv("data/transactions.csv")

In [ ]:
print("There are",len(items_df['family'].unique()),"families of products or items")
print("There are",len(store_df['type'].unique()),"type of stores")
print("Stores are in ",len(store_df['city'].unique()),"cities in ", len(store_df['state'].unique()),"states")

### Load holiday information 
1. assign the national holoday to all the items at cetain date
2. assign the local holiday to item at certain city at certain date
3. no correspnondling regional holiday info, ignore

In [ ]:
# load holiday info
holiday_df = pd.read_csv('data/holidays_events.csv',parse_dates=['date'],dtype= {'transferred':np.bool})
holiday_df = holiday_df.query('transferred == False')
holiday_df=holiday_df.drop(['transferred','description'],axis = 1)
# subdivide holiday information
national_holiday_df = holiday_df[holiday_df.locale == 'National']
national_holiday_df.drop(['locale'],axis = 1)
regional_holiday_df = holiday_df[holiday_df.locale == 'Regional']
regional_holiday_df.drop(['locale'],axis = 1)
local_holiday_df = holiday_df[holiday_df.locale == 'Local']
local_holiday_df.drop(['locale'],axis = 1)
# check the holiday distribution
# only need to merge city for local holiday
unique_locale = np.unique(local_holiday_df.locale_name)
unique_city = np.unique(store_df.city)
unique_state = np.unique(store_df.state)

print ([k if k in unique_city else '' for k in unique_locale])
print ([k if k in unique_state else '' for k in unique_locale])

## Preliminary Feature Reduction

refer to https://www.kaggle.com/sohinibhattacharya86/predict-grocery-sales-rf-xgb

#### Question 1 - Is there any statistically significant relation between Store Type and Cluster of the stores ?

1. Null Hypothesis H0 = Store Type (a, b, c, d, e) and Cluster (1 to 17) are independent from each other.
2. Alternative Hypothesis HA = Store Tpe and cluster are not independent of each other. There is a relationship between them.
3. Store Type - categorical variable
4. Cluster - categorical variable

Now, to determine if there is a statistically significant correlation between the variables, we use a chi-square test of independence of variables in a contingency table

Here, we create a contingency table, with the frequencies of all possible values

** Conclusion ** cluster is depend on store type, so drop store type to reduce the dimension

In [ ]:
# Contingency table
ct = pd.crosstab(store_df['type'], store_df['cluster'])
display(ct)

#### Question 2 - Is there any statistically significant relation between Promotion and unit scales ?

1. Null Hypothesis H0 = Promotion and Sales are independent from each other.
2. Alternative Hypothesis HA = Promotion and Sales are not independent of each other. There is a relationship between them.
3. Promotion - categorical variable - Independent variable
4. Sales - continuous variable - Dependent variable

Now, to determine if there is a statistically significant correlation between the variables, we use a student t test

2-sample t-test: testing for difference across populations

** Conclusion ** There is a relationship between onpromotion and sales

In [ ]:
promo_sales = train_df[train_df['onpromotion'] == 1.0]['unit_sales']
nopromo_sales = train_df[train_df['onpromotion'] == 0.0]['unit_sales']
st.ttest_ind(promo_sales, nopromo_sales, equal_var = False)

#### Question 3 - Is there any statistically significant relation between Oil price and Sales of the stores ?

1. Null Hypothesis H0 = Oil price and Sales are independent from each other.
2. Alternative Hypothesis HA = Oil price and Sales are not independent of each other. There is a relationship between them.
3. Oil Price - Independent continuous variable
4. Sales - Dependent continuous variable

conlusion: ** ??? ** (Drop first)

## Joining multiple features
- drop: 
1. oil.csv, because we are only dealing with 2017, so assume oil price does not have so quick influence
2. cluster in store (efficiency reason)
- disregard for now: transections.csv

#### version1: nearly all joint

In [ ]:
# join the store information
train = pd.merge(train_df, store_df, on= "store_nbr",how = 'left')
# join the item feature
train = pd.merge(train, items_df, on= "item_nbr",how = 'left')
# join holiday info
train = pd.merge(train,national_holiday_df[['date', 'type']],on = 'date', how='left', suffixes=['', '_nationalHoliday'])
train = pd.merge(train, local_holiday_df[['date','type','locale_name']], 
                 left_on = ['date','city'], right_on=['date','locale_name'], 
                 how='left', suffixes=['', '_cityHoliday'])
train = train.drop(['locale_name'], axis=1)

## Feature Engineering

In [ ]:
train.loc[(train.unit_sales<0),'unit_sales'] = 0 
train['unit_sales_cal'] =  train['unit_sales'].apply(pd.np.log1p) 

train['type_nationalHoliday'] = train['type_nationalHoliday'].fillna('no')
train['type_cityHoliday'] = train['type_cityHoliday'].fillna('no')
train['type_nationalHoliday'] = train['type_nationalHoliday'].replace({'Additional': 'Holiday','Transfer':'Holiday','Additional':'Holiday'})
train['type_cityHoliday'] = train['type_cityHoliday'].replace({'Additional': 'Holiday','Transfer':'Holiday','Additional':'Holiday'})

train['type_nationalHoliday'] = train['type_nationalHoliday'].astype('category')
train['type_cityHoliday'] = train['type_cityHoliday'].astype('category')
train['onpromotion'] = train['onpromotion'].astype('category')
train['city'] = train['city'].astype('category')

cat_columns = train.select_dtypes(['category']).columns
train[cat_columns] = train[cat_columns].apply(lambda x: x.cat.codes)
train.head(1)

## Testing (see training section below)

1. according to the training below we choose
    - Gradient Boosting: max_depth = 12
    - Random Forest: max_depth = 12
    - AdaBoost + Decisin Tree: max_depth = 12, n_estimator = 6

#### Define the loss function

In [ ]:
def cost_function(y, pred, w):
    from sklearn import metrics
    return metrics.mean_squared_error(y, pred, sample_weight=w)**0.5

#### define the training feature

In [ ]:
from sklearn.cross_validation import train_test_split
import operator

X_fea = [c for c in list(train.columns.values) if c not in ['id','unit_sales','unit_sales_cal','class','date']]
Y_fea = ['unit_sales','unit_sales_cal']
display(X_fea)
display(Y_fea)

#### train all the data using settled model

In [ ]:
#xgb
import xgboost as xgb
import time
def rmspe_xg(pred, y):
    y = y.get_label()
    return "rmspe", np.sqrt(np.mean((pred-y) ** 2))
gb_train = {'mean':[],'var':[]}; gb_val = {'mean':[],'var':[]}; gb_time = []
X_Y_train, X_Y_val = train_test_split(train, test_size=0.2, random_state=0)
dtrain = xgb.DMatrix(X_Y_train[X_fea], X_Y_train[Y_fea].unit_sales_cal)
dvalid = xgb.DMatrix(X_Y_val[X_fea], X_Y_val[Y_fea].unit_sales_cal)
params = {"objective": "reg:linear","booster" : "gbtree","eta": 0.3,"max_depth": 13,"subsample": 0.9,"colsample_bytree": 0.7,
          "silent": 0,"seed": 0}
num_boost_round = 30
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, \
  early_stopping_rounds=10, feval=rmspe_xg, verbose_eval=True)
del train;
del watchlist; del dtrain;del dvalid;

#### Plot the validation set (because ground trutch of testing set is not avalilabel on kaggle)

In [ ]:
X_Y_val2=X_Y_val.head(100)
pred =  gbm.predict(xgb.DMatrix(X_Y_val2[X_fea]))
fig = plt.figure(figsize=(25,10))
ax1 = plt.subplot(231)
ax1.plot(X_Y_val2[X_fea].day,pred,'ro',label = 'pred')
ax1.plot(X_Y_val2[X_fea].day.head(1000),X_Y_val2[Y_fea].unit_sales_cal,'g^',label = 'ground_truth')
ax1.set_xlabel('day')
ax1.set_ylabel('logp1(unit_sales)')
ax1.legend()
ax1.set_title('day vs logp1(unit_sales)')

ax2 = plt.subplot(232)
ax2.plot(X_Y_val2[X_fea].store_nbr,pred,'ro',label = 'pred')
ax2.plot(X_Y_val2[X_fea].store_nbr,X_Y_val2[Y_fea].unit_sales_cal, 'g^',label = 'ground_truth')
ax2.legend()
ax2.set_xlabel('store_nbr')
ax2.set_ylabel('logp1(unit_sales)')
ax2.set_title('store_nbr vs logp1(unit_sales)')


ax3 = plt.subplot(233)
ax3.plot(X_Y_val2[X_fea].family,pred,'ro',label = 'pred')
ax3.plot( X_Y_val2[X_fea].family,X_Y_val2[Y_fea].unit_sales_cal,'g^',label = 'ground_truth')
ax3.legend()
ax3.set_xlabel('family')
ax3.set_ylabel('logp1(unit_sales)')
ax3.set_title('family vs logp1(unit_sales)')


ax4 = plt.subplot(234)
ax4.plot(X_Y_val2[X_fea].item_nbr,pred,'ro',label = 'pred')
ax4.plot(X_Y_val2[X_fea].item_nbr,X_Y_val2[Y_fea].unit_sales_cal,'g^',label = 'ground_truth')
ax4.legend()
ax4.set_xlabel('item_nbr')
ax4.set_ylabel('logp1(unit_sales)')
ax4.set_title('item number vs logp1(unit_sales)')

ax5 = plt.subplot(235)
ax5.plot(X_Y_val2[X_fea].month,pred,'ro',label = 'pred')
ax5.plot(X_Y_val2[X_fea].month,X_Y_val2[Y_fea].unit_sales_cal,'g^',label = 'ground_truth')
ax5.legend()
ax5.set_xlabel('month')
ax5.set_ylabel('logp1(unit_sales)')
ax5.set_title('month vs logp1(unit_sales)')

plt.show()

In [ ]:
#rf
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=0,max_depth=12,n_jobs=-1)
regr.fit(train[X_fea], train[Y_fea].unit_sales_cal.values)


In [ ]:
# Adaboost
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

ada_train = {'mean':[],'var':[]}; ada_val = {'mean':[],'var':[]}
X_fea2 = [c for c in X_fea if c not in ['type_cityHoliday','perishable','state','quater','onpromotion']]
print('features {}',X_fea2)

regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=12),n_estimators=6, random_state=0)
regr.fit(train[X_fea2], train[Y_fea].unit_sales_cal.values)


#### Do the testing

In [ ]:


## CAUTION!! : load full data
# load complete train data
types_dict = {'id': 'int32',
             'item_nbr': 'int32',
             'store_nbr': 'int8',
             'unit_sales': 'float32',
             'onpromotion': 'bool'}

test_df = pd.read_csv('data/test.csv',low_memory=True,  parse_dates=['date'],dtype=types_dict)
# join the store information
test = pd.merge(test_df, store_df, on= "store_nbr",how = 'left')
# join the item feature
test = pd.merge(test, items_df, on= "item_nbr",how = 'left')
# join holiday info
test = pd.merge(test,national_holiday_df[['date', 'type']],on = 'date', how='left', suffixes=['', '_nationalHoliday'])
test = pd.merge(test, local_holiday_df[['date','type','locale_name']], 
                 left_on = ['date','city'], right_on=['date','locale_name'], 
                 how='left', suffixes=['', '_cityHoliday'])
test = test.drop(['locale_name'], axis=1)

test['type_nationalHoliday'] = test['type_nationalHoliday'].fillna('no')
test['type_cityHoliday'] = test['type_cityHoliday'].fillna('no')
test['type_nationalHoliday'] = test['type_nationalHoliday'].replace({'Additional': 'Holiday','Transfer':'Holiday','Additional':'Holiday'})
test['type_cityHoliday'] = test['type_cityHoliday'].replace({'Additional': 'Holiday','Transfer':'Holiday','Additional':'Holiday'})

test['type_nationalHoliday'] = test['type_nationalHoliday'].astype('category')
test['type_cityHoliday'] = test['type_cityHoliday'].astype('category')
test['onpromotion'] = test['onpromotion'].astype('category')
test['city'] = test['city'].astype('category')

cat_columns = test.select_dtypes(['category']).columns
test[cat_columns] = test[cat_columns].apply(lambda x: x.cat.codes)
del store_df; del items_df;
del national_holiday_df; del local_holiday_df;
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['quater'] = test['date'].dt.quarter
test.head(1)

test['unit_sales'] = gbm.predict(xgb.DMatrix(test[X_fea]))
# test['unit_sales'] = regr.predict(test[X_fea])
# test['unit_sales'] = regr.predict(test[X_fea2])
test['unit_sales'] = np.expm1(test['unit_sales'])
test[['id','unit_sales']].to_csv('gb_pred', index=False, float_format='%.5f')
# test[['id','unit_sales']].to_csv('ada_pred', index=False, float_format='%.5f')


# Training

Reference: https://www.kaggle.com/lscoelho/stacking-model-lm-etr-rf-and-gbr


In [ ]:
cur_train = train; del train

#### CV on Gradient Boostting
Here I use xgboost instead of the gradietn boosting in sklearn because they are basically the same method but sklearn's implemnet is much slower

refer to https://www.kaggle.com/sohinibhattacharya86/predict-grocery-sales-rf-xgb

In [ ]:
import xgboost as xgb
import time
def rmspe(y,pred):
    return np.sqrt(np.mean((pred-y) ** 2))
def rmspe_xg(pred, y):
    y = y.get_label()
    return "rmspe", np.sqrt(np.mean((pred-y) ** 2))
gb_train = {'mean':[],'var':[]}; gb_val = {'mean':[],'var':[]}; gb_time = []
for max_depth in range(8,12): 
    tmp_train = [];tmp_val = [];tmp_time = []
    for k in range(0,3):
        X_Y_train, X_Y_val = train_test_split(cur_train, test_size=0.2, random_state=int(11*k))
        
        dtrain = xgb.DMatrix(X_Y_train[X_fea], X_Y_train[Y_fea].unit_sales_cal)
        dvalid = xgb.DMatrix(X_Y_val[X_fea], X_Y_val[Y_fea].unit_sales_cal)
        params = {"objective": "reg:linear","booster" : "gbtree","eta": 0.3,"max_depth": max_depth,"subsample": 0.9,"colsample_bytree": 0.7,
                  "silent": 0,"seed": 0}
        num_boost_round = 30
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, \
          early_stopping_rounds=10, feval=rmspe_xg, verbose_eval=True)
        pred = gbm.predict(xgb.DMatrix(X_Y_train[X_fea]))
        train_error = cost_function(X_Y_train[Y_fea].unit_sales_cal, pred, X_Y_train.perishable.map({False: 1.0, True: 1.25}))
        print('GB - Performance: NWRMSLE(1) = ',train_error)
        pred = gbm.predict(xgb.DMatrix(X_Y_val[X_fea]))
        val_error = cost_function(X_Y_val[Y_fea].unit_sales_cal,pred,X_Y_val.perishable.map({False: 1.0, True: 1.25}))
        print('GB - Performance: NWRMSLE(1) = ',val_error)
        del X_Y_train, X_Y_val
        tmp_train.append(train_error)
        tmp_val.append(val_error)
    gb_train['mean'].append(np.mean(tmp_train)); gb_train['var'].append(np.var(tmp_train))
    gb_val['mean'].append(np.mean(tmp_val)); gb_val['var'].append(np.var(tmp_val))

#### CV on Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
# rf_train = {'mean':[],'var':[]}; rf_val = {'mean':[],'var':[]}; gb_time = []
for max_depth in range(12,13): 
    tmp_train = [];tmp_val = [];tmp_time = []
    for k in range(0,3):
        X_Y_train, X_Y_val = train_test_split(cur_train, test_size=0.2, random_state=int(11*k))
        
        regr = RandomForestRegressor(random_state=0,max_depth=max_depth,n_jobs=-1)
        regr.fit(X_Y_train[X_fea], X_Y_train[Y_fea].unit_sales_cal.values)
        train_error = cost_function(X_Y_train[Y_fea].unit_sales_cal, regr.predict(X_Y_train[X_fea]), X_Y_train.perishable.map({False: 1.0, True: 1.25}))
        print('RF - Performance: NWRMSLE(1) = ',train_error)
        val_error = cost_function(X_Y_val[Y_fea].unit_sales_cal,regr.predict(X_Y_val[X_fea]),X_Y_val.perishable.map({False: 1.0, True: 1.25}))
        print('RF - Performance: NWRMSLE(1) = ',val_error)
        del X_Y_train, X_Y_val
        tmp_train.append(train_error)
        tmp_val.append(val_error)
    rf_train['mean'].append(np.mean(tmp_train)); rf_train['var'].append(np.var(tmp_train))
    rf_val['mean'].append(np.mean(tmp_val)); rf_val['var'].append(np.var(tmp_val))

In [ ]:
print(rf_train)
print(rf_val)

#### CV on AdaBoost

1. use the feature importance analysis above to simply the feature, in order to relieve the efficiency burden
    - threshold: relative importance < 20%

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_train = {'mean':[],'var':[]}; dt_val = {'mean':[],'var':[]}
X_fea2 = [c for c in X_fea if c not in ['type_cityHoliday','perishable','state','quater','onpromotion']]
print('features {}',X_fea2)

for max_depth in range(9,13): 
    tmp_train = [];tmp_val = [];tmp_time = []
    for k in range(0,3):
        X_Y_train, X_Y_val = train_test_split(cur_train, test_size=0.2, random_state=int(11*k))
        regr = DecisionTreeRegressor(max_depth=max_depth)
        regr.fit(X_Y_train[X_fea2], X_Y_train[Y_fea].unit_sales_cal.values)
        train_error = cost_function(X_Y_train[Y_fea].unit_sales_cal, regr.predict(X_Y_train[X_fea2]), X_Y_train.perishable.map({False: 1.0, True: 1.25}))
        print('Adaboost+Decision Tree - Performance: NWRMSLE(1) = ',train_error)
        val_error = cost_function(X_Y_val[Y_fea].unit_sales_cal,regr.predict(X_Y_val[X_fea2]),X_Y_val.perishable.map({False: 1.0, True: 1.25}))
        print('Adaboost+Decision Tree - Performance: NWRMSLE(1) = ',val_error)
        del X_Y_train, X_Y_val
        tmp_train.append(train_error)
        tmp_val.append(val_error)
    dt_train['mean'].append(np.mean(tmp_train)); dt_train['var'].append(np.var(tmp_train))
    dt_val['mean'].append(np.mean(tmp_val)); dt_val['var'].append(np.var(tmp_val))

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

ada_train = {'mean':[],'var':[]}; ada_val = {'mean':[],'var':[]}
X_fea2 = [c for c in X_fea if c not in ['type_cityHoliday','perishable','state','quater','onpromotion']]
print('features {}',X_fea2)

for n_estimators in range(2,16,2): 
    tmp_train = [];tmp_val = [];tmp_time = []
    for k in range(0,3):
        X_Y_train, X_Y_val = train_test_split(cur_train, test_size=0.2, random_state=int(11*k))
        regr = AdaBoostRegressor(DecisionTreeRegressor(max_depth=12),n_estimators=n_estimators, random_state=0)
        regr.fit(X_Y_train[X_fea2], X_Y_train[Y_fea].unit_sales_cal.values)
        train_error = cost_function(X_Y_train[Y_fea].unit_sales_cal, regr.predict(X_Y_train[X_fea2]), X_Y_train.perishable.map({False: 1.0, True: 1.25}))
        print('Adaboost+Decision Tree - Performance: NWRMSLE(1) = ',train_error)
        val_error = cost_function(X_Y_val[Y_fea].unit_sales_cal,regr.predict(X_Y_val[X_fea2]),X_Y_val.perishable.map({False: 1.0, True: 1.25}))
        print('Adaboost+Decision Tree - Performance: NWRMSLE(1) = ',val_error)
        del X_Y_train, X_Y_val
        tmp_train.append(train_error)
        tmp_val.append(val_error)
    ada_train['mean'].append(np.mean(tmp_train)); ada_train['var'].append(np.var(tmp_train))
    ada_val['mean'].append(np.mean(tmp_val)); ada_val['var'].append(np.var(tmp_val))

In [ ]:
print(ada_train)
print(ada_val)

#### visualize functions

In [ ]:
# Plot feature importance

def plotFeaforDict(feature_importance_dict):
    # make importances relative to max importance
    feature_importance = list(feature_importance_dict.values())
    names = list(feature_importance_dict.keys())
    feature_importance = 100.0 * (feature_importance / np.max(feature_importance))
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5
    plt.barh(pos, feature_importance[sorted_idx], align='center')
    plt.yticks(pos, [names[i] for i in sorted_idx])
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')
    plt.show()
    
plotFeaforDict(gbm.get_fscore())